In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def generate_Vector_P_Norm(p=1, vec_size=5, min_value=-100, max_value=100):  # Generate Lp norm for a given vector, assuming p = 1, and assuming user wants positives only
    x_vect = []
    for i in range(vec_size):
        x_vect.append(np.random.uniform(min_value, max_value))  # Random vector is based on the distribution specified by the user.

    y_value = np.power(np.sum(np.power(np.abs(x_vect), p)), (1/p)) # Generate answer for P norm
    return x_vect, y_value


def create_dataset(no_examples=10, p=1, vec_size=5, min_value=-100, max_value=100):
    X, y = [], []
    for _ in range(no_examples):
        x_vect, y_value = generate_Vector_P_Norm(p, vec_size, min_value, max_value)
        X.append(x_vect)
        y.append(y_value)
    return np.array(X), np.array(y)

def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment(no_examples=100, no_experiments=5, vec_size=5, p=1, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset(no_examples=i, p=p, vec_size=vec_size, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            model = build_model((vec_size,))
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            loss, mae = model.evaluate(X_test, y_test, verbose=0)
            mse_experiment.append(loss)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example



In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

def run_sgd_experiment(no_examples=100, no_experiments=5, vec_size=5, p=1, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset(no_examples=i, p=p, vec_size=vec_size, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            # Standardize the features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Create and fit the SGD regressor
            sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3)
            sgd_regressor.fit(X_train, y_train)

            # Predict and calculate errors
            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))

            mse_experiment.append(mse)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example

In [ ]:
mse_results, mae_results, rmse_results = run_experiment(no_examples=100, no_experiments=50, vec_size=5, p = 2,  min_value=-100, max_value=100)
np.savetxt('neural_network_mse_results_0.5P_Norm_50exp_100ex_vec5_MSE.txt', mse_results)
np.savetxt('neural_network_mse_results_0.5P_Norm_50exp_100ex_vec5_MAE.txt', mae_results)
np.savetxt('neural_network_mse_results_0.5P_Norm_50exp_100ex_vec5_RMSE.txt', rmse_results)

'''
plt.plot(range(3, 101), mse_results, label='Neural Network')
plt.xlabel('Number of Examples')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Neural Network Performance on L2 Norm Tasks')
plt.legend()
plt.show()
'''

Examples: 3, MSE: 36205.59140625, MAE: 175.9192610168457, RMSE: 190.27766922644918
Examples: 4, MSE: 30662.110239257814, MAE: 164.87379089355468, RMSE: 175.1059971538891
Examples: 5, MSE: 25579.1280078125, MAE: 149.6988475036621, RMSE: 159.93476172431212
Examples: 6, MSE: 25422.873850097658, MAE: 148.0149831390381, RMSE: 159.4455200063572
Examples: 7, MSE: 25219.264271850585, MAE: 142.05800521850585, RMSE: 158.805743825123
Examples: 8, MSE: 22526.134826660156, MAE: 135.72341011047362, RMSE: 150.0870908061721
Examples: 9, MSE: 19415.261740722657, MAE: 126.47797187805176, RMSE: 139.3386584574527
Examples: 10, MSE: 22178.60660644531, MAE: 134.89243606567382, RMSE: 148.92483542527523
Examples: 11, MSE: 20286.3078125, MAE: 126.87722877502442, RMSE: 142.43001022432034
Examples: 12, MSE: 15891.56859375, MAE: 114.32561546325684, RMSE: 126.06176499537835
Examples: 13, MSE: 15368.803486328125, MAE: 112.5246492767334, RMSE: 123.97097840352848
Examples: 14, MSE: 8331.30850341797, MAE: 74.985480918

"\nplt.plot(range(3, 101), mse_results, label='Neural Network')\nplt.xlabel('Number of Examples')\nplt.ylabel('Mean Squared Error (MSE)')\nplt.title('Neural Network Performance on L2 Norm Tasks')\nplt.legend()\nplt.show()\n"

In [ ]:
mse_results, mae_results, rmse_results = run_sgd_experiment(no_examples=100, no_experiments=50, vec_size=5, p=2, min_value=-100, max_value=100)
np.savetxt('sgd_mse_results_0.5P_Norm_50exp_100ex_vec5_MSE.txt', mse_results)
np.savetxt('sgd_mae_results_0.5P_Norm_50exp_100ex_vec5_MAE.txt', mae_results)
np.savetxt('sgd_rmse_results_0.5P_Norm_50exp_100ex_vec5_RMSE.txt', rmse_results)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 3, MSE: 7318.443219499728, MAE: 52.9719992265729, RMSE: 85.5479001466414
SGD - Examples: 4, MSE: 4172.137935486538, MAE: 50.89178135233691, RMSE: 64.59208879953131


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 5, MSE: 2972.790556077072, MAE: 35.58935956148395, RMSE: 54.52330287204795
SGD - Examples: 6, MSE: 2667.530879345618, MAE: 35.863376670594334, RMSE: 51.64814497487415


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

...

SGD - Examples: 21, MSE: 1248.327594862171, MAE: 27.641664156326748, RMSE: 35.33167976281585
SGD - Examples: 22, MSE: 1095.9189464767594, MAE: 26.119586657885765, RMSE: 33.10466653625678


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 23, MSE: 1291.0984163908968, MAE: 27.38734363276299, RMSE: 35.93185795907159
SGD - Examples: 24, MSE: 1118.5928157422404, MAE: 26.486047535084477, RMSE: 33.44537061750461


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 25, MSE: 1115.642209638575, MAE: 26.499673627131283, RMSE: 33.40123066053967
SGD - Examples: 26, MSE: 1195.728915898498, MAE: 26.8580973200498, RMSE: 34.57931340987698
SGD - Examples: 27, MSE: 969.1745597880837, MAE: 24.86230612502147, RMSE: 31.13156854043952
SGD - Examples: 28, MSE: 1044.634182643412, MAE: 25.90534521643286, RMSE: 32.32080108294675
SGD - Examples: 29, MSE: 1032.7869608865867, MAE: 25.34471498745441, RMSE: 32.13700298544634
SGD - Examples: 30, MSE: 953.5596374690437, MAE: 24.517794668405433, RMSE: 30.879760968457052
SGD - Examples: 31, MSE: 1047.0526903529394, MAE: 25.36605081351807, RMSE: 32.35819355824641
SGD - Examples: 32, MSE: 988.0374213420515, MAE: 25.346600295036996, RMSE: 31.433062551110915
SGD - Examples: 33, MSE: 913.6538951192139, MAE: 23.20412140253292, RMSE: 30.226708307707174
SGD - Examples: 34, MSE: 904.0391233566335, MAE: 23.634181127354626, RMSE: 30.067243361449574
SGD - Examples: 35, MSE: 872.8810401733847, MAE: 23.634554140129225, RM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def generate_Matrix_P_Norm(n=2, m=2, p=2, min_value=-100, max_value=100):
    # Generate a random matrix of size n x m with values between min_value and max_value
    matrix = np.random.uniform(min_value, max_value, (n, m))

    # Compute the p-norm (entrywise)
    p_norm = np.sum(np.abs(matrix) ** p) ** (1 / p)

    return matrix.flatten(), p_norm  # Flatten the matrix to return it as a vector

def create_dataset(no_examples=10, n=2, m=2, p=2, min_value=-100, max_value=100):
    X, y = [], []
    for _ in range(no_examples):
        x_vect, y_value = generate_Matrix_P_Norm(n, m, p, min_value, max_value)
        X.append(x_vect)
        y.append(y_value)
    return np.array(X), np.array(y)

def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment(no_examples=100, no_experiments=5, n=2, m=2, p=2, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset(no_examples=i, n=n, m=m, p=p, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            model = build_model((n * m,))
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            loss, mae = model.evaluate(X_test, y_test, verbose=0)
            mse_experiment.append(loss)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example

In [ ]:
mse_results, mae_results, rmse_results = run_experiment(p = 0.5, no_examples=100, no_experiments=50, n=5, m=5, min_value=-100, max_value=100)

# Saving the results to files
np.savetxt('neural_network_mse_results_PNorm_50exp_100ex_2x2_MSE.txt', mse_results)
np.savetxt('neural_network_mae_results_PNorm_50exp_100ex_2x2_MAE.txt', mae_results)
np.savetxt('neural_network_rmse_results_PNorm_50exp_100ex_2x2_RMSE.txt', rmse_results)

Examples: 3, MSE: 725566694.4, MAE: 26766.43921875, RMSE: 26936.34523093287
Examples: 4, MSE: 825482608.0, MAE: 28514.880703125, RMSE: 28731.21313136638
Examples: 5, MSE: 786099885.44, MAE: 27835.207109375, RMSE: 28037.47287898822
Examples: 6, MSE: 791386957.44, MAE: 27823.5796484375, RMSE: 28131.600691037827
Examples: 7, MSE: 777569589.76, MAE: 27597.918125, RMSE: 27884.934817209094
Examples: 8, MSE: 802637072.64, MAE: 28118.000625, RMSE: 28330.850192678652
Examples: 9, MSE: 792755710.08, MAE: 27857.1457421875, RMSE: 28155.91785184777
Examples: 10, MSE: 780448778.88, MAE: 27671.9061328125, RMSE: 27936.513362980713
Examples: 11, MSE: 774926049.92, MAE: 27548.3056640625, RMSE: 27837.49359981965
Examples: 12, MSE: 786146654.72, MAE: 27752.7044921875, RMSE: 28038.30691607466
Examples: 13, MSE: 774847138.56, MAE: 27549.00265625, RMSE: 27836.076206247173
Examples: 14, MSE: 754038369.28, MAE: 27105.04125, RMSE: 27459.75908998475
Examples: 15, MSE: 771785298.56, MAE: 27510.9439453125, RMSE: 2

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

def run_sgd_experiment(no_examples=100, no_experiments=5, n=2, m=2, p=2, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset(no_examples=i, n=n, m=m, p=p, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            # Standardize the features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3)
            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))

            mse_experiment.append(mse)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example

In [ ]:
mse_results, mae_results, rmse_results = run_sgd_experiment(p = 2, no_examples=100, no_experiments=50, n=5, m=5, min_value=-100, max_value=100)

np.savetxt('sgd_mse_results_PNorm_50exp_100ex_2x2_MSE.txt', mse_results)
np.savetxt('sgd_mae_results_P_50exp_100ex_2x2_MAE.txt', mae_results)
np.savetxt('sgd_rmse_results_P_50exp_100ex_2x2_RMSE.txt', rmse_results)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 3, MSE: 3954.197195292847, MAE: 45.64702343061719, RMSE: 62.88240767728958
SGD - Examples: 4, MSE: 1332.7690968985744, MAE: 30.945239322424058, RMSE: 36.507110223880694


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 5, MSE: 1308.456299468355, MAE: 29.1565261620765, RMSE: 36.17259044453901
SGD - Examples: 6, MSE: 1147.518956913279, MAE: 26.69709378849692, RMSE: 33.875049179496095
SGD - Examples: 7, MSE: 1169.040571797357, MAE: 27.353573480306636, RMSE: 34.19123530668872
SGD - Examples: 8, MSE: 1050.0808245899088, MAE: 25.738880861317742, RMSE: 32.40495061853835
SGD - Examples: 9, MSE: 1408.6484473019782, MAE: 28.793336138318104, RMSE: 37.53196567330278
SGD - Examples: 10, MSE: 1378.8444642487912, MAE: 28.54260320463975, RMSE: 37.13279499645551
SGD - Examples: 11, MSE: 984.9747439867384, MAE: 25.529151465523572, RMSE: 31.384307288623376
SGD - Examples: 12, MSE: 840.2417214610649, MAE: 23.782194160064456, RMSE: 28.98692328380273
SGD - Examples: 13, MSE: 1438.4274584144782, MAE: 30.46908924784473, RMSE: 37.92660620744332
SGD - Examples: 14, MSE: 1296.9135748597475, MAE: 28.510297269229525, RMSE: 36.012686304408724


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


...

SGD - Examples: 50, MSE: 1778.306356089789, MAE: 33.757374381579055, RMSE: 42.16996983743039


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 51, MSE: 1833.048117932667, MAE: 34.28334368975617, RMSE: 42.81411120101254


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 52, MSE: 1630.639984940191, MAE: 31.945599032357205, RMSE: 40.38118355051262
SGD - Examples: 53, MSE: 1717.105919958672, MAE: 32.687796745807574, RMSE: 41.437976784088676
SGD - Examples: 54, MSE: 1698.499389541958, MAE: 31.832585762236512, RMSE: 41.2128546638298
SGD - Examples: 55, MSE: 1675.6617803046777, MAE: 32.39302604722345, RMSE: 40.93484799415624
SGD - Examples: 56, MSE: 1536.6963047369313, MAE: 31.022192429809948, RMSE: 39.20071816608634
SGD - Examples: 57, MSE: 1582.848305049089, MAE: 31.406598332437117, RMSE: 39.785026141113555
SGD - Examples: 58, MSE: 1346.2436933615268, MAE: 28.733940121622616, RMSE: 36.691193675888044
SGD - Examples: 59, MSE: 1465.1059875241308, MAE: 30.472779269608285, RMSE: 38.276702934345465
SGD - Examples: 60, MSE: 1473.045566089211, MAE: 30.413629881664118, RMSE: 38.38027574274592
SGD - Examples: 61, MSE: 1425.0018303074196, MAE: 29.225614237033266, RMSE: 37.74919641935997
SGD - Examples: 62, MSE: 1277.2248142573358, MAE: 28.6858225290

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def generate_Matrix_Nuclear_Norm(n=2, m=2, min_value=-100, max_value=100):
    # Generate a random matrix of size n x m with values between min_value and max_value
    matrix = np.random.uniform(min_value, max_value, (n, m))

    # Compute the nuclear norm, which is the sum of the singular values of the matrix
    singular_values = np.linalg.svd(matrix, compute_uv=False)
    nuclear_norm = np.sum(singular_values)

    return matrix.flatten(), nuclear_norm  # Flatten the matrix to return it as a vector

def create_dataset_nuclear_norm(no_examples=10, n=2, m=2, min_value=-100, max_value=100):
    X, y = [], []
    for _ in range(no_examples):
        x_vect, y_value = generate_Matrix_Nuclear_Norm(n, m, min_value, max_value)
        X.append(x_vect)
        y.append(y_value)
    return np.array(X), np.array(y)

def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_nuclear_norm(no_examples=100, no_experiments=5, n=2, m=2, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset_nuclear_norm(no_examples=i, n=n, m=m, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            model = build_model((n * m,))
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            loss, mae = model.evaluate(X_test, y_test, verbose=0)
            mse_experiment.append(loss)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example


In [ ]:
mse_results, mae_results, rmse_results = run_experiment_nuclear_norm(no_examples=100, no_experiments=50, n=5, m=5, min_value=-100, max_value=100)

# Saving the results to files
np.savetxt('neural_network_mse_results_NuclearNorm_50exp_100ex_5x5_MSE.txt', mse_results)
np.savetxt('neural_network_mae_results_NuclearNorm_50exp_100ex_5x5_MAE.txt', mae_results)
np.savetxt('neural_network_rmse_results_NuclearNorm_50exp_100ex_5x5_RMSE.txt', rmse_results)


Examples: 3, MSE: 186510.74640625, MAE: 425.2024526977539, RMSE: 431.8688995589402
Examples: 4, MSE: 155687.473671875, MAE: 383.36749450683595, RMSE: 394.57252016818785
Examples: 5, MSE: 148855.853359375, MAE: 373.72468139648436, RMSE: 385.81842019190196
Examples: 6, MSE: 144986.565625, MAE: 370.7342541503906, RMSE: 380.77101468599204
Examples: 7, MSE: 137777.16625, MAE: 359.2848574829102, RMSE: 371.18346710218657
Examples: 8, MSE: 127194.5644921875, MAE: 343.9299682617187, RMSE: 356.6434697175703
Examples: 9, MSE: 123729.324921875, MAE: 338.0028500366211, RMSE: 351.75179448280716
Examples: 10, MSE: 110904.368125, MAE: 319.47380889892577, RMSE: 333.023074463317
Examples: 11, MSE: 98959.180625, MAE: 299.4706282043457, RMSE: 314.5777815183393
Examples: 12, MSE: 106814.8858203125, MAE: 311.7206832885742, RMSE: 326.82546690904076
Examples: 13, MSE: 92795.28740234375, MAE: 288.27179428100584, RMSE: 304.6231892065076
Examples: 14, MSE: 55560.08310546875, MAE: 217.2821565246582, RMSE: 235.711

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

def run_sgd_experiment_nuclear_norm(no_examples=100, no_experiments=5, n=2, m=2, min_value=-100, max_value=100):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    for i in range(3, no_examples + 1):  # Starting with 3 prior examples
        mse_experiment = []
        mae_experiment = []

        for _ in range(no_experiments):
            X, y = create_dataset_nuclear_norm(no_examples=i, n=n, m=m, min_value=min_value, max_value=max_value)

            if i > 2:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            else:
                X_train, y_train = X, y
                X_test, y_test = X, y

            # Standardize the features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3)
            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))

            mse_experiment.append(mse)
            mae_experiment.append(mae)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.sqrt(mse_mean)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    return mse_per_example, mae_per_example, rmse_per_example

In [ ]:
mse_results, mae_results, rmse_results = run_sgd_experiment_nuclear_norm(no_examples=100, no_experiments=50, n=5, m=5, min_value=-100, max_value=100)

np.savetxt('sgd_mse_results_NuclearNorm_50exp_100ex_5x5_MSE.txt', mse_results)
np.savetxt('sgd_mae_results_NuclearNorm_50exp_100ex_5x5_MAE.txt', mae_results)
np.savetxt('sgd_rmse_results_NuclearNorm_50exp_100ex_5x5_RMSE.txt', rmse_results)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 3, MSE: 1911726.7970682362, MAE: 332.03483842485963, RMSE: 1382.6520882232942
SGD - Examples: 4, MSE: 7941.815517655082, MAE: 69.19942536111483, RMSE: 89.11686438410567


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 5, MSE: 6857.771794546831, MAE: 65.49630424949468, RMSE: 82.81166460437099
SGD - Examples: 6, MSE: 5848.159843375395, MAE: 62.09401737441414, RMSE: 76.47326227757905
SGD - Examples: 7, MSE: 6627.100813604493, MAE: 64.49282794314855, RMSE: 81.40700715297481
SGD - Examples: 8, MSE: 6354.345498080591, MAE: 61.83534214184349, RMSE: 79.71414866935851
SGD - Examples: 9, MSE: 6363.922951094323, MAE: 64.74348666034086, RMSE: 79.7741997834784
SGD - Examples: 10, MSE: 4693.420931776798, MAE: 53.01155866978873, RMSE: 68.50854641412849
SGD - Examples: 11, MSE: 5372.541648226193, MAE: 57.85522191269998, RMSE: 73.29762375565932
SGD - Examples: 12, MSE: 4934.902413858831, MAE: 54.28958435085415, RMSE: 70.24886058761972
SGD - Examples: 13, MSE: 6741.859827111324, MAE: 65.55024982575132, RMSE: 82.10882916660866
SGD - Examples: 14, MSE: 7357.512777462376, MAE: 69.44438543600305, RMSE: 85.77594521462515
SGD - Examples: 15, MSE: 6962.931393969434, MAE: 66.97174665220564, RMSE: 83.444181306

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 31, MSE: 15530.853518324395, MAE: 98.932564385934, RMSE: 124.622845089993


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 32, MSE: 14989.705195568358, MAE: 97.70956087513925, RMSE: 122.43245156235481


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

#SINGULAR VALUE TESTING
##INCLUDES THE FOLLOWING IN ORDER
###INITIALIZATION OF DENSE 2 LAYER NN FOLLOWED BT TESTING OF DENSE 2 LAYER NN
###INITIALIZATION OF MULTI REGRESSION SGD FOLLOWED BT TESTING OF MULTI REGRESSION SGD
###INITIALIZATION OF CNN FOLLOWED BY TESTING OF CNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def generate_Top_k_Singular_Values(n=2, m=2, k=2, min_value=-100, max_value=100):
    # Generate a random matrix of size n x m with values between min_value and max_value
    matrix = np.random.uniform(min_value, max_value, (n, m))

    # Compute the singular values of the matrix
    singular_values = np.linalg.svd(matrix, compute_uv=False)

    # Ensure k does not exceed the number of singular values
    k = min(k, len(singular_values))

    # Extract the top k singular values
    top_k_singular_values = singular_values[:k]

    return matrix, top_k_singular_values  # Return the matrix and the top k singular values

def create_dataset(no_examples=10, n=2, m=2, k=2, min_value=-100, max_value=100):
    X, y = [], []
    matrices = []  # To store the list of matrices
    for _ in range(no_examples):
        matrix, y_value = generate_Top_k_Singular_Values(n, m, k, min_value, max_value)
        matrices.append(matrix)  # Store matrix
        X.append(matrix.flatten())  # Flatten matrix to use as input
        y.append(y_value)
    return np.array(X), np.array(y), matrices

def build_model(input_shape, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(output_dim, activation='linear')  # Adjust the output dimension to match the top k values
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment(no_examples=100, no_experiments=5, n=2, m=2, k=2, min_value=-100, max_value=100):
    mse_per_example = np.zeros(no_examples - 2)
    rmse_per_example = np.zeros(no_examples - 2)
    mae_per_example = np.zeros(no_examples - 2)

    # DataFrame to store experiment results
    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    for i in range(3, no_examples + 1):  # Start with 3 examples and go up to 100
        mse_experiment = []
        mae_experiment = []
        j = 0

        for _ in range(no_experiments):
            # Create dataset with `i` examples
            X, y, matrices = create_dataset(no_examples=i, n=n, m=m, k=k, min_value=min_value, max_value=max_value)

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            # Create model with output dimension equal to k
            model = build_model((n * m,), k)
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            # Predict the top k singular values for the last matrix
            y_pred = model.predict(X_test)
            loss = np.mean((y_test - y_pred) ** 2)  # Calculate MSE manually
            mae = np.mean(np.abs(y_test - y_pred))

            mse_experiment.append(loss)
            mae_experiment.append(mae)

            # Calculate RMSE
            rmse = np.sqrt(loss)

            # Store results in DataFrame
            temp_df = pd.DataFrame({
                'i': [i],
                'j': [j],
                'input_matrices': [matrices],  # Store all matrices used in this experiment
                'actual_answers': [y_test.flatten()],
                'predicted_answers': [y_pred.flatten()],
                'mse': [loss],
                'rmse': [rmse],
                'mae': [mae]
            })

            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

            # Update cumulative errors for averaging later
            mse_per_example[i - 3] += loss
            rmse_per_example[i - 3] += rmse
            mae_per_example[i - 3] += mae
            j += 1

        print(f"Examples: {i}, MSE: {np.mean(mse_experiment)}, MAE: {np.mean(mae_experiment)}, RMSE: {np.sqrt(np.mean(mse_experiment))}")

    # Average out error metrics across all experiments
    mse_per_example /= no_experiments
    rmse_per_example /= no_experiments
    mae_per_example /= no_experiments

    # Create a summary DataFrame for the average results
    df_average_results = pd.DataFrame({
        'i': range(3, no_examples + 1),
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
#TOP K DENSE
df_experiment_results, df_average_results = run_experiment(
    no_examples=100,
    no_experiments=50,
    n=5,
    m=5,
    k=3,
    min_value=-100,
    max_value=100
)

df_experiment_results.to_csv('2NN_experiment_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)

df_average_results.to_csv('2NN_average_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)


1/1 [==============================] - 0s 51ms/step
Examples: 3, MSE: 17224.0259952181, MAE: 121.16255139388812, RMSE: 131.24033676891455


<ipython-input-16-9e01f9a11cb5>:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


1/1 [==============================] - 0s 50ms/step
Examples: 4, MSE: 8455.968445551889, MAE: 88.91528252715555, RMSE: 91.95633988775265
1/1 [==============================] - 0s 51ms/step
Examples: 5, MSE: 7905.533638078966, MAE: 86.88249451309905, RMSE: 88.9130678701335
1/1 [==============================] - 0s 51ms/step
Examples: 6, MSE: 12563.800192115335, MAE: 106.5494969571423, RMSE: 112.08835886083503
1/1 [==============================] - 0s 52ms/step
Examples: 7, MSE: 16881.02120426403, MAE: 120.7536538999546, RMSE: 129.92698412671646
1/1 [==============================] - 0s 56ms/step
Examples: 8, MSE: 4669.526762632655, MAE: 59.45632829557915, RMSE: 68.33393565888515
1/1 [==============================] - 0s 51ms/step
Examples: 9, MSE: 1723.7330921451849, MAE: 40.29391982777544, RMSE: 41.51786473489677
1/1 [==============================] - 0s 52ms/step
Examples: 10, MSE: 5841.419385356333, MAE: 65.18095001359701, RMSE: 76.42917888710001
1/1 [==============================] 

In [ ]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd


# WE USE MULTI OUTPUT REGRESSOR SINCE SGD CANNOT HANDLE A MULTI OUTPUT Y FOR EACH INPUT
# IT CAN ONLY HANDLE A SINGLE VALUE Y FOR A MULTI DIMENSIONAL INPUT X
# MultiOutputRegressor tries to predict each output y separately linearly.


def generate_Top_k_Singular_Values(n=2, m=2, k=2, min_value=-100, max_value=100):
    # Generate a random matrix of size n x m with values between min_value and max_value
    matrix = np.random.uniform(min_value, max_value, (n, m))

    # Compute the singular values of the matrix
    singular_values = np.linalg.svd(matrix, compute_uv=False)

    # Ensure k does not exceed the number of singular values
    k = min(k, len(singular_values))

    # Extract the top k singular values
    top_k_singular_values = singular_values[:k]

    return matrix.flatten(), top_k_singular_values

def create_dataset(no_examples=10, n=2, m=2, k=2, min_value=-100, max_value=100):
    X, y = [], []
    matrices = []  # To store the list of matrices
    for _ in range(no_examples):
        x_vect, y_value = generate_Top_k_Singular_Values(n, m, k, min_value, max_value)
        matrices.append(x_vect.reshape(n, m))  # Store the original matrix (not flattened)
        X.append(x_vect)
        y.append(y_value)
    return np.array(X), np.array(y), matrices

def run_sgd_experiment_top_k_singular_values(no_examples=100, no_experiments=5, n=2, m=2, k=2, min_value=-100, max_value=100):
    # Initialize lists to store average error metrics for each example size
    mse_per_example = np.zeros(no_examples - 2)
    rmse_per_example = np.zeros(no_examples - 2)
    mae_per_example = np.zeros(no_examples - 2)

    # DataFrame to store experiment results
    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    for i in range(2, no_examples + 1):  # Start with 3 examples and go up to 100
        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for j in range(no_experiments):
            # Create dataset with `i` examples
            X, y, matrices = create_dataset(no_examples=i, n=n, m=m, k=k, min_value=min_value, max_value=max_value)

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Using MultiOutputRegressor with SGDRegressor to handle multi-output y
            sgd_regressor = MultiOutputRegressor(SGDRegressor(max_iter=10000, tol=1e-3))
            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            # Store only the top k values as actual answers
            actual_answers_top_k = [y_test.flatten()[:k]]
            predicted_answers_top_k = [y_pred.flatten()[:k]]

            # Store experiment details in DataFrame
            temp_df = pd.DataFrame({
                'i': [i],
                'j': [j],
                'input_matrices': [matrices],  # Store all matrices used in this experiment
                'actual_answers': actual_answers_top_k,
                'predicted_answers': predicted_answers_top_k,
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        # Calculate average errors for each example size
        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example[i - 3] = mse_mean
        mae_per_example[i - 3] = mae_mean
        rmse_per_example[i - 3] = rmse_mean

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    # Create summary DataFrame for average results
    df_average_results = pd.DataFrame({
        'i': range(3, no_examples + 1),
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results

In [ ]:
df_experiment_results, df_average_results = run_sgd_experiment_top_k_singular_values(
    no_examples=100,
    no_experiments=50,
    n=5,
    m=5,
    k=3,
    min_value=-100,
    max_value=100
)

df_experiment_results.to_csv('SGD_experiment_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)

df_average_results.to_csv('SGD_average_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)

<ipython-input-19-cc0b9ac36348>:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


SGD - Examples: 2, MSE: 869.6143096715317, MAE: 23.72688757695189, RMSE: 27.1714102160035
SGD - Examples: 3, MSE: 305295.4036412569, MAE: 160.2859670810183, RMSE: 194.49930034876564
SGD - Examples: 4, MSE: 893.2331361304066, MAE: 23.67117195773217, RMSE: 26.94421401316486
SGD - Examples: 5, MSE: 744.3534480083333, MAE: 21.933734020982214, RMSE: 25.316139116965825
SGD - Examples: 6, MSE: 696.8842374612914, MAE: 21.584597827944748, RMSE: 24.903249529729777
SGD - Examples: 7, MSE: 994.5888772513089, MAE: 24.360489908610706, RMSE: 27.984888142566135
SGD - Examples: 8, MSE: 922.813226857376, MAE: 23.435209250845283, RMSE: 27.179020092138344
SGD - Examples: 9, MSE: 785.7377749478561, MAE: 22.47344995309041, RMSE: 25.77717796917583
SGD - Examples: 10, MSE: 1132.1089345409086, MAE: 26.919037907028198, RMSE: 30.830305923929195
SGD - Examples: 11, MSE: 934.6067016158895, MAE: 24.079563067745767, RMSE: 27.719711849028695
SGD - Examples: 12, MSE: 845.3968566012987, MAE: 23.172456730331366, RMSE: 2

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 23, MSE: 3193.8097110136086, MAE: 43.15262658540843, RMSE: 49.59014543719917


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 30, MSE: 3452.7255252835603, MAE: 45.65336029740504, RMSE: 52.86076158670514


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 31, MSE: 2908.2763785401703, MAE: 41.84156873867115, RMSE: 48.22061874491962
SGD - Examples: 32, MSE: 2049.2429662296267, MAE: 35.3143853827564, RMSE: 40.605072922002826
SGD - Examples: 33, MSE: 2047.4417809215804, MAE: 35.36489028966261, RMSE: 41.370227692248555
SGD - Examples: 34, MSE: 1723.552769201824, MAE: 32.34145646847697, RMSE: 37.893015675459004
SGD - Examples: 35, MSE: 2234.0564282331284, MAE: 37.14242637991118, RMSE: 41.94296688573978
SGD - Examples: 36, MSE: 1142.0619385523553, MAE: 26.606826866414785, RMSE: 30.86801173998189
SGD - Examples: 37, MSE: 1684.0611036955079, MAE: 32.6429798759352, RMSE: 37.26319405858308
SGD - Examples: 38, MSE: 1459.5324931401926, MAE: 29.798857616205442, RMSE: 34.78321094337052
SGD - Examples: 39, MSE: 1411.3628821874015, MAE: 31.20531151736251, RMSE: 35.2987936988744
SGD - Examples: 40, MSE: 1123.980079494101, MAE: 26.55687219568509, RMSE: 30.486923733107883
SGD - Examples: 41, MSE: 1587.0988493605475, MAE: 33.065522377493814,

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

def generate_Top_k_Singular_Values(n=2, m=2, k=2, min_value=-100, max_value=100):
    # Generate a random matrix of size n x m with values between min_value and max_value
    matrix = np.random.uniform(min_value, max_value, (n, m))

    # Compute the singular values of the matrix
    singular_values = np.linalg.svd(matrix, compute_uv=False)

    # Ensure k does not exceed the number of singular values
    k = min(k, len(singular_values))

    # Extract the top k singular values
    top_k_singular_values = singular_values[:k]

    return matrix, top_k_singular_values

def create_dataset(no_examples=10, n=2, m=2, k=2, min_value=-100, max_value=100):
    X, y = [], []
    matrices = []  # To store the list of matrices
    for _ in range(no_examples):
        x_mat, y_value = generate_Top_k_Singular_Values(n, m, k, min_value, max_value)
        matrices.append(x_mat)  # Store the original matrix
        X.append(x_mat)
        y.append(y_value)
    return np.array(X), np.array(y), matrices

def build_cnn_model(input_shape, k):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(k, activation='linear')
    ])
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_top_k_singular_values_cnn(no_examples=100, no_experiments=5, n=2, m=2, k=2, min_value=-100, max_value=100):
    # Initialize arrays to store average error metrics for each example size
    mse_per_example = np.zeros(no_examples - 2)
    rmse_per_example = np.zeros(no_examples - 2)
    mae_per_example = np.zeros(no_examples - 2)

    # DataFrame to store detailed experiment results
    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    for i in range(3, no_examples + 1):  # Start with 3 examples and go up to 100
        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for j in range(no_experiments):
            # Create dataset with `i` examples
            X, y, matrices = create_dataset(no_examples=i, n=n, m=m, k=k, min_value=min_value, max_value=max_value)

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, n, m)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            # Expand dimensions for CNN input
            X_train = np.expand_dims(X_train, axis=-1)
            X_test = np.expand_dims(X_test, axis=-1)

            model = build_cnn_model((n, m, 1), k)
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            # Store only the top k values as actual answers
            actual_answers_top_k = [y_test.flatten()[:k]]
            predicted_answers_top_k = [y_pred.flatten()[:k]]

            # Store experiment details in DataFrame
            temp_df = pd.DataFrame({
                'i': [i],
                'j': [j],
                'input_matrices': [matrices],  # Store all matrices used in this experiment
                'actual_answers': actual_answers_top_k,
                'predicted_answers': predicted_answers_top_k,
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        # Calculate average errors for each example size
        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example[i - 3] = mse_mean
        mae_per_example[i - 3] = mae_mean
        rmse_per_example[i - 3] = rmse_mean

        print(f"CNN - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    # Create summary DataFrame for average results
    df_average_results = pd.DataFrame({
        'i': range(3, no_examples + 1),
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results

In [ ]:
df_experiment_results, df_average_results = run_experiment_top_k_singular_values_cnn(
    no_examples=100,
    no_experiments=50,
    n=5,
    m=5,
    k=3,
    min_value=-100,
    max_value=100
)

df_experiment_results.to_csv('cnn_experiment_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)

df_average_results.to_csv('cnn_average_results_TopKSingularValues_50exp_100ex_5x5.csv', index=False)

1/1 [==============================] - 0s 68ms/step


<ipython-input-20-772ba3a64595>:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


1/1 [==============================] - 0s 62ms/step


KeyboardInterrupt: 

###THE FOLLOWING IS TESTING BASED OFF THE SAME DATA WE TESTED THE ICL ON. WE SAVED THE DATA SET AND WANT TO PREDICT THE SV's USING TH SAME MATRICES

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(input_shape, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(output_dim, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_with_predefined_data(df, n=2, m=2, k=2):
    # Sort and reset index
    df = df.sort_values(by=['i', 'j']).reset_index(drop=True)

    unique_i_values = df['i'].unique()
    #print("Unique 'i' values (number of prior examples) in dataframe:", unique_i_values)

    mse_per_example = np.zeros(len(unique_i_values))
    rmse_per_example = np.zeros(len(unique_i_values))
    mae_per_example = np.zeros(len(unique_i_values))

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    for idx, i in enumerate(unique_i_values):  # Correctly loop through unique 'i' values
        #print(f"Processing for i={i} (number of prior examples)")
        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        # Filter the dataframe to get rows where 'i' matches
        experiments_for_i = df[df['i'] == i]

        # Print the filtered dataframe for debugging
        #print(f"Experiments for i={i}:\n", experiments_for_i)

        # Ensure we only have 50 experiments for each 'i'
        if len(experiments_for_i) != 50:
            print(f"Warning: Number of experiments for i={i} is {len(experiments_for_i)} instead of 50.")

        for j in range(50):  # Process exactly 50 experiments per 'i'
            row = experiments_for_i.iloc[j]

            try:
                # Safely evaluate using numpy's array function
                env = {"np": np, "array": np.array}
                questions = eval(row['raw_questions'], env)
                answers = np.array(eval(row['raw_answers'], env))
            except (SyntaxError, ValueError, NameError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            # Prepare training and testing data
            X_train = np.array([q.flatten() for q in questions[:-1]])  # All but last question
            y_train = answers[:-1]  # All but last answer
            X_test = questions[-1].flatten().reshape(1, -1)  # Last question as test
            y_test = answers[-1].reshape(1, -1)  # Last answer as test

            model = build_model((n * m,), k)
            model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [questions],  # Store all matrices used in this experiment
                'actual_answers': [y_test.flatten()],
                'predicted_answers': [y_pred.flatten()],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })

            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

            print(f"Examples: {row['i']}, Experiment: {row['j']}, MSE: {mse}, MAE: {mae}, RMSE: {rmse}")

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example[idx] = mse_mean
        mae_per_example[idx] = mae_mean
        rmse_per_example[idx] = rmse_mean

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
df = pd.read_csv('LLM_results_Top_K_Matrix_25x25_k3.txt_k_experiment_results.csv')
df.tail()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_experiment_results, df_average_results = run_experiment_with_predefined_data(df, n=10, m=10, k=3)

df_experiment_results_path = '/content/drive/My Drive/10x10_k3_2NN_df_experiment_results.csv'
df_average_results_path = '/content/drive/My Drive/10x10_k3_2NN_df_average_results.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'run_experiment_with_predefined_data' is not defined

In [ ]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import pandas as pd

def run_sgd_experiment_with_predefined_data(df, n=2, m=2, k=2):
    # Sort and reset index
    df = df.sort_values(by=['i', 'j']).reset_index(drop=True)

    unique_i_values = df['i'].unique()

    mse_per_example = np.zeros(len(unique_i_values))
    rmse_per_example = np.zeros(len(unique_i_values))
    mae_per_example = np.zeros(len(unique_i_values))

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    for idx, i in enumerate(unique_i_values):
        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        experiments_for_i = df[df['i'] == i]

        # Ensure we only have 50 experiments for each 'i'
        if len(experiments_for_i) != 50:
            print(f"Warning: Number of experiments for i={i} is {len(experiments_for_i)} instead of 50.")

        for j in range(50):  # Process exactly 50 experiments per 'i'
            row = experiments_for_i.iloc[j]

            try:
                env = {"np": np, "array": np.array}
                questions = eval(row['raw_questions'], env)
                answers = np.array(eval(row['raw_answers'], env))
            except (SyntaxError, ValueError, NameError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X_train = np.array([q.flatten() for q in questions[:-1]])  # All but last question
            y_train = answers[:-1]  # All but last answer
            X_test = questions[-1].flatten().reshape(1, -1)  # Last question as test
            y_test = answers[-1].reshape(1, -1)  # Last answer as test

            scaler = RobustScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Using MultiOutputRegressor with SGDRegressor to handle multi-output y
            sgd_regressor = MultiOutputRegressor(SGDRegressor(max_iter=1000, tol=1e-3, eta0=0.001, learning_rate='adaptive'))

            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [questions],  # Store all matrices used in this experiment
                'actual_answers': [y_test.flatten()],
                'predicted_answers': [y_pred.flatten()],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })

            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

            print(f"Examples: {row['i']}, Experiment: {row['j']}, MSE: {mse}, MAE: {mae}, RMSE: {rmse}")

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example[idx] = mse_mean
        mae_per_example[idx] = mae_mean
        rmse_per_example[idx] = rmse_mean

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_experiment_results, df_average_results = run_sgd_experiment_with_predefined_data(df, n=25, m=25, k=3)

df_experiment_results_path = '/content/drive/My Drive/25x25_k3_SGD_df_experiment_results_new.csv'
df_average_results_path = '/content/drive/My Drive/25x25_k3_SGD_df_average_results_new.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Examples: 2, Experiment: 0, MSE: 26089.672807886836, MAE: 160.91402790643176, RMSE: 161.52297919456177
Examples: 2, Experiment: 1, MSE: 39992.43788042813, MAE: 197.1941037439718, RMSE: 199.98109380746004


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
<ipython-input-45-f6b257403bbd>:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries 

Examples: 2, Experiment: 2, MSE: 29313.852036692417, MAE: 171.11195941085074, RMSE: 171.21288513628994
Examples: 2, Experiment: 3, MSE: 31898.72372748291, MAE: 177.86690674682973, RMSE: 178.6021380820591
Examples: 2, Experiment: 4, MSE: 38631.15629765479, MAE: 196.0007312924562, RMSE: 196.548101740146


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

Examples: 2, Experiment: 5, MSE: 29892.18192768336, MAE: 170.14592822491733, RMSE: 172.89355664015753
Examples: 2, Experiment: 6, MSE: 28782.572774303306, MAE: 169.5935787989162, RMSE: 169.6542742588683
Examples: 2, Experiment: 7, MSE: 41184.66968875751, MAE: 202.68910679791358, RMSE: 202.94006427701137


...

Examples: 3, Experiment: 48, MSE: 4084.4255439518715, MAE: 63.39767853095143, RMSE: 63.909510590771006
Examples: 3, Experiment: 49, MSE: 39710.17939961222, MAE: 151.46434207381876, RMSE: 199.27413128555403
Examples: 4, Experiment: 0, MSE: 39823.03483151948, MAE: 198.26116599197442, RMSE: 199.55709667040028


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Examples: 4, Experiment: 1, MSE: 409260.8966569079, MAE: 638.058501266062, RMSE: 639.7350206584816
Examples: 4, Experiment: 2, MSE: 2058.2629754769164, MAE: 42.24200284966594, RMSE: 45.368083224629586
Examples: 4, Experiment: 3, MSE: 80597.30805717154, MAE: 283.26916126890984, RMSE: 283.8966503098822
Examples: 4, Experiment: 4, MSE: 29035.32738027051, MAE: 167.55360465809517, RMSE: 170.3975568494763
Examples: 4, Experiment: 5, MSE: 33297.89588884903, MAE: 181.45273698214461, RMSE: 182.47711058883257
Examples: 4, Experiment: 6, MSE: 27649.20722086533, MAE: 164.82485853446698, RMSE: 166.28050763954664
Examples: 4, Experiment: 7, MSE: 44690.67971602808, MAE: 207.34252229828147, RMSE: 211.40170225432925
Examples: 4, Experiment: 8, MSE: 194382.98616542693, MAE: 440.2631143116166, RMSE: 440.8888591985818
Examples: 4, Experiment: 9, MSE: 19376551.79178694, MAE: 4388.531267194288, RMSE: 4401.88048358732
Examples: 4, Experiment: 10, MSE: 284.0734335507195, MAE: 12.860081429403843, RMSE: 16.8544

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Examples: 6, Experiment: 35, MSE: 5.7841488126821813e+20, MAE: 19730924055.055332, RMSE: 24050257405.446167
Examples: 6, Experiment: 36, MSE: 2406477.33794043, MAE: 1022.9475100562903, RMSE: 1551.2824816713523
Examples: 6, Experiment: 37, MSE: 2106146903.2942421, MAE: 45185.1692140706, RMSE: 45892.77615588582
Examples: 6, Experiment: 38, MSE: 14062827602523.037, MAE: 3726090.1616786644, RMSE: 3750043.6800820115
Examples: 6, Experiment: 39, MSE: 350835.8030095618, MAE: 592.0750797554454, RMSE: 592.3139395705302
Examples: 6, Experiment: 40, MSE: 726436.8578049155, MAE: 851.4301710595195, RMSE: 852.3126526134148
Examples: 6, Experiment: 41, MSE: 1261699.5416596495, MAE: 1122.0083438891018, RMSE: 1123.2539969479965
Examples: 6, Experiment: 42, MSE: 125105.59859217283, MAE: 352.6426183208605, RMSE: 353.70269802783923
Examples: 6, Experiment: 43, MSE: 2440125440.1717496, MAE: 35575.95572656533, RMSE: 49397.62585561931
Examples: 6, Experiment: 44, MSE: 2373362595431.4478, MAE: 956108.21508180

In [ ]:
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

def build_cnn_model(input_shape, k):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(k, activation='linear')
    ])
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_with_predefined_data_cnn(df, n=2, m=2, k=2):
    mse_per_example = []
    rmse_per_example = []
    mae_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for j, row in experiments_for_i.iterrows():
            try:
                env = {"np": np, "array": np.array}
                matrices = eval(row['raw_questions'], env)
                answers = np.array(eval(row['raw_answers'], env))
            except (SyntaxError, ValueError, NameError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array(matrices)
            y = answers

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, n, m)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            X_train = np.expand_dims(X_train, axis=-1)
            X_test = np.expand_dims(X_test, axis=-1)

            # Build and train CNN model
            model = build_cnn_model((n, m, 1), k)
            model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            # Predict and calculate error metrics
            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            # Store only the top k values as actual answers
            actual_answers_top_k = [y_test.flatten()[:k]]
            predicted_answers_top_k = [y_pred.flatten()[:k]]

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [matrices],
                'actual_answers': actual_answers_top_k,
                'predicted_answers': predicted_answers_top_k,
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"CNN - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
df_experiment_results, df_average_results = run_experiment_with_predefined_data_cnn(df, n=10, m=10, k=3)

df_experiment_results_path = '/content/drive/My Drive/10x10_k3_CNN_df_experiment_results.csv'
df_average_results_path = '/content/drive/My Drive/10x10_k3_CNN_df_average_results.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Unique 'i' values in dataframe: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100]
Processing for i=2 (number of prior examples)
1/1 [==============================] - 0s 101ms/step


<ipython-input-6-6a7d1c38f387>:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 64ms/step
CNN - Examples: 4, MSE: 799.6755465957167, MAE: 22.590310060380734, RMSE: 26.268216667665747
Processing for i=5 (number of prior examples)
1/1 [==============================] - 0s 64ms/step
CNN - Examples: 5, MSE: 762.4292499723924, MAE: 21.689767871872924, RMSE: 25.290215113791625
Processing for i=6 (number of prior examples)
1/1 [==============================] - 0s 65ms/step
CNN - Examples: 6, MSE: 854.5956612909758, MAE: 22.88330505004531, RMSE: 26.61260372671296
Processing for i=7 (number of prior examples)
1/1 [==============================] - 0s 63ms/step
CNN - Examples: 7, MSE: 712.6684446228167, MAE: 21.62080270491171, RMSE: 24.876846162207325
Processing for i=8 (number of prior examples)
1/1 [==============================] - 0s 69ms/step
CNN - Examples: 8, MSE: 620.6022425644846, MAE: 19.450583341502345, RMSE: 22.65439026399266
Processing for i=9 (number of prior examples